<a href="https://colab.research.google.com/github/AlbertoBassanoni/MLPNS_ABassanoni/blob/main/MLPNS23_Lesson_30_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering:

Come abbiamo visto l'altra volta, vi sono diverse definizioni di distanze utilizzate negli algoritmi di clustering. Scegliendo nelle distanze di Minkowski un diverso valore di p, si ha che ciò influisce differentemente sul trattamento degli errori e la composizione dei miei dati. 

Tra le diverse tipologie di distanze ci sono anche delle distanze categoriche, che si descrivono attraverso delle variabili binarie (true or false), e come si può fare clustering con questa tipologia di variabili? Bisogna definire un'opportuna distanza per questo tipo di problemi!

Quali sono i principali desiderata per i clustering?

-Scalabilità
-Abilità di avere a che fare con diversi tipi di attributi
-Scoperta di cluster con shape arbitraria
-Minime condizioni iniziali e conoscenza sul dominio (pochi hyperparameters)
-Gestione del noise e degli outliers 
-Insensibili all'ordine
-Ammette l'incorporazione di vincoli
-Interpretabile

ESEMPIO:

Che vuol dire fare clustering sull'image processing? Si può prendere un'immagine, composta da pixel, in generale composizione di tre colori, e posso costruire un algorimo che dia una distinzione dei colori in base alla luminosità della foto. Ridurre la dimensionalità in un algoritmo di clustering significa fondamentalmente rendere la feature space più semplice (ed approssimata), che è descrivibile con meno parametri. Algoritmi di clustering servono ad esempio in urban science per tracciare il numero di luci in una foto di New York, e tramite altri algoritmi calcolare quanta energia elettrica serve per alimentare quelle luci. 

# Whitening:

Il termine whitening si riferisce al white noise, cioè del noise che ha la stessa intensità ad ogni spettro di frequenze (non ci sono range spettrali preferenziali). Tutti i dataset possono avere delle covarianze o delle correlazioni tra i vari dati. Ad esempio, un dataset sulle previsioni metereologiche variabili quali umidità, temperatura, precipitazioni hanno un grado di covarianza che li lega tra di loro. 

Ciò è un problema, poiché la maggior parte dei teoremi che conosciamo si basano su un'ipotesi di non correlazione tra la variabili. Una variabile correlata ad una variabile esogena cambia proporzionalmente al cambiare di quella esogena. Può accadere per due motivi: c'è una vera correlazione, ad esempio il GDP e il numero di abitanti di un paese, essendo gli abitanti il driver dell'economia (si può parlare di causalità); non c'è una reale correlazione tra le due variabili, ma sono entrambe influenzate da una terza variabili, e si parla di correlazioni spurie, ad esempio il numero di gelati comprati e le morti per affogamento. Il driver vero è la temperatura. (non si può parlare di causalità);

Bisogna fare attenzione ad una cosa importante: ALTA CORRELAZIONE =\ CAUSALITA'! 

Genericamente la correlazione si esprime attraverso la Pearson correlation ( o linear correlation), data dal prodotto degli errori delle due variabili diviso il prodotto delle loro deviazioni standard:

**$r_{xy}=\frac{\sum_{i=1}^n (x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^n (x_i-\bar{x})^2}\sqrt{\sum_{i=1}^n (y_i-\bar{y})^2}}$**

Ora diventiamo un po' più pratici. Se le variabili sono correlate, si distribuiscono su una linea. Io vorrei ottenere un cerchio, e per farlo dovrei diagonalizzare la matrice di covarianza, e si sa che è un processo lungo in quanto le matrici sono grosse e non sempre la diagonalizzazione è possibile. Esiste tuttavia un metodo che approssima i dati correlati a dei dati non correlati, in particolare questo si chiama algoritmo ZCA, e questo processo di "de-correlazione" dei dati si chiama whitening.

Genericamente non utilizzeremo ZCA, ma trasformeremo questo set di dati riscalandoli in una distribuzione gaussiana a media zero e varianza uno. Spesso utilizzeremo la funzione preprocessing.scale, e si può specificare su una singola variabile (un solo asse) o su tutto lo spazio delle variabili. Nelle time series non si può utilizzare il preprocessing su tutte le variabili, perché vogliamo tenere inalterata la forma delle time series!

X=preprocessing.scale(X, axis=0)

Tutte le colonne del vettore X hanno media 0 e std pari a 1

OGNI VOLTA bisogna sempre fare il preprocessing con questa funzione!



# K-Means:

Il primo modello di clustering è un modello di hard partitioning, chiamato modello K-Means. Hard vuol dire che l'assegnamento dei punti del cluster non è probabilistico, ma deterministico ("questo punto appartiene al cluster 1" NON "è probabile che..."). Questo algoritmo crea delle regioni in cui appartengono i dati.

L'algoritmo funziona così:

- Scelgo N centri random nel mio feature space. N è il numero dei miei clusters (è un hyperparameter). 

- Per ogni punto del feature space, calcolo la distanza di ogni punto da ciascuno degli N centri, e assegno quel punto al centro avente distanza minore;

Perché si chiama K-means? Perché dopo aver scelto i miei centri a caso, la posizione del centro del cluster si sposta, perché diventa la media del cluster "sferico" che si viene a formare. Questo algoritmo continua a calcolare i new cluster centers fino alla convergenza, ovverosia i cluster, ridefiniti iterativamente, a un certo punto non si muovono più. 

Quale potrebbe essere il criterio di convergenza del K-means? Quando la posizione del centro non cambia più di una certa soglia **$\epsilon$**.

Quali sono i problemi di questo algoritmo?

- Devo sapere quanti cluster voglio! Conoscenza che a priori potrei non avere senza ulteriori informazioni!

- Funziona solo con le variabili numeriche!

- La soluzione non è deterministica ed univoca, poiché dipende esplicitamente da dove scelgo i miei initial guess random per i centri!


Vantaggio di questo algoritmo è che scala con un ordine O(K d N), cioè scala linearmente in funzione del numero dei clusters K, la dimensione delle variabili d, e il numero di variabili N;


Questo algoritmo si basa sulla minimizzazione della distanza aggregata tra i cluster. La intra-cluster variance totale che va minimizzata è:

**$\sum_k \sum_{i \in k} (x_i - \mu_k)^2$**

Dove **$x_i$** è la posizione del punto e **$\mu_l$** è la posizione del centro.


#DB - SCAN:

Abbiamo visto il KMeans nell'esempio sulle luminosità delle stelle. è sensato questo tipo di clustering?Oppure esiste un più intelligente insieme
di clustering in 3 gruppi? Sì, esiste! KMeans non lo può fare però, perché
KMeans si basa solo sulla distanza bidimensionale! Ma allora i cluster che si trovano con KMeans saranno tutti clusters basati sulla distanza, e quindi sui blobs sferici, e non riconosce dei vuoti all'interno del nostro feature space.

Un altro algoritmo sarebbe quello di non calcolare solo la distanza rispetto al
centro, ma anche la distanza tra i punti a due a due, per costruire un algoritmo di clustering affinity based (ma ci dà una distanza d^2, probematica in termini di costo computazionale)

Il DB - SCAN è il clustering algorithm più utilizzato, e sta per Density Based spacial clustering of applications with noise (noise nel senso di outliers). Questo è uno degli algoritmi di clustering più comuni e dei più citati in letteratura scientifica. 

Il cluster center stavolta è definito su densità locale, ovverosia se ha abbastanza vicini per poter stabilire un concetto di densità (si basa sull'algoritmo dei Nearest Neibourghs, che vedremo).


L'algoritmo funziona così:

- il punto p è un core point se ha un certo numero di vicini superiore ad una certa threshold;

- il punto q è directly reachable da p core point se la distanza d(q,p) è inferiore ad una distanza soglia **$\epsilon$**;

- il punto s è reachable da p se esiste un cammino (p1,... pn) da p1=p e pn=s dove p(i) è directly reachable a p(i+1)

- il punto t che non soddisfa alcuna di queste ocndizioni è not reachable, e quindi non appartiene al cluster;

Gli hyperparameters sono:

- **$\epsilon$** massima distanza dai centri;
- min_sample: numero minimo di punti in un cluster;
- metric: tipo di metrica con cui calcolare la distanza;
- p: se scelgo la metrica Minkowskiana, dichiaro il grado della metrica Lp;

L'ordine dell'algoritmo è di tipo O(K, d, N^2), con N numero dei punti. I pro di questo algoritmo sono che:

- E' deterministico (devo solo fissare i miei hyperparameters);
- E' capace di identificare outliers;
- Può essere utilizzato con diverse definizioni di distanza;

I contro sono:

- Non è interamente deterministico (devo comunque assegnare dei valori iniziali);
- Si basa sull'ipotesi di densità costante;

